In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20180630_MO_Pax7_Cece/wm/migration'

In [6]:
# Add experiment date here to apply to dataframe
expt_date = '20180630'
treatment = 'SMPD3MO'
result_type = 'Migration'

path = os.path.abspath('')+'/pax7migrationCSV/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    df['Target'], df['Treatment'], df['Embryo'], df['Mag'], df['Somites'] = zip(*df['Image'].map(lambda x: x.split('_')))
    
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,Angle,IntDen,RawIntDen,Length,Date,Image,delete1,ROI,delete2,delete3,Target,Treatment,Embryo,Mag,Somites
0,1,A:CntlArea:c:1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,131356.284,2151.087,0.000,2.825588e+08,1.370873e+09,0.000,20180630,SMPD3_MO_Emb4_10x_8ss,A,CntlArea,c,1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,SMPD3,MO,Emb4,10x,8ss
1,2,A:ExptArea:c:1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,83570.144,2701.794,0.000,2.257893e+08,1.095448e+09,0.000,20180630,SMPD3_MO_Emb4_10x_8ss,A,ExptArea,c,1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,SMPD3,MO,Emb4,10x,8ss
2,3,A:Cntl:c:1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,126.349,2069.289,-88.877,2.614528e+05,1.268474e+06,277.901,20180630,SMPD3_MO_Emb4_10x_8ss,A,Cntl,c,1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,SMPD3,MO,Emb4,10x,8ss
3,4,A:Expt:c:1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,88.012,2589.332,-88.790,2.278911e+05,1.105645e+06,193.447,20180630,SMPD3_MO_Emb4_10x_8ss,A,Expt,c,1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,SMPD3,MO,Emb4,10x,8ss
4,5,A:Cntl:c:1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,90.485,2320.758,-89.215,2.099936e+05,1.018813e+06,198.871,20180630,SMPD3_MO_Emb4_10x_8ss,A,Cntl,c,1/2 - SMPD3_MO_Emb4_10x_8ss.czi #1,SMPD3,MO,Emb4,10x,8ss


In [7]:
# Mean length values and group
area_meanLength = (full_df.groupby(['Embryo', 'ROI'])['Area', 'Length']).mean()

# Extract Areas, generate ratios
area_cntl = area_meanLength.xs('CntlArea', level='ROI')['Area'] 
area_expt = area_meanLength.xs('ExptArea', level='ROI')['Area'] 

# Generate ratios as Expt/Cntl
area_ratios = pd.DataFrame(area_expt / area_cntl)
area_ratios.columns = ['Expt/Cntl Area']

# Extract lengths, generate ratios
length_cntl = area_meanLength.xs('Cntl', level='ROI')['Length']
length_expt = area_meanLength.xs('Expt', level='ROI')['Length'] 

# Generate ratios as Expt/Cntl
length_ratios = pd.DataFrame(length_expt / length_cntl)
length_ratios.columns = ['Expt/Cntl Length']

In [9]:
# Combine processed values into single dataframe and output as csv file 'Results.csv'
area_cntl = pd.DataFrame(area_cntl)
area_cntl.columns = ['Cntl Area']
area_expt = pd.DataFrame(area_expt)
area_expt.columns = ['Expt Area']

length_cntl = pd.DataFrame(length_cntl)
length_cntl.columns = ['Cntl Length']
length_expt = pd.DataFrame(length_expt)
length_expt.columns = ['Expt Length']


results = pd.concat([area_cntl, area_expt, area_ratios, length_cntl, length_expt, length_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index + '_Cece'
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')